## Setup libraries

In [ ]:
%env MPICH_GPU_SUPPORT_ENABLED=0

import sys

import numpy as np
import pickle as pkl

from astropy.table import Table
from tqdm import tqdm
from scipy.optimize import fsolve

from numcosmo_py import nc
from numcosmo_py import ncm

__name__ = "NcContext"

ncm.cfg_init()
ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

## Load cluster catalog and bin definitions

In [ ]:
cluster_catalog = Table.read("hamana_clusters.fits")
pz_bins = Table.read("pz/s16a_pz_pdf_bins.fits")

## Create `data_cluster_wl` and `likelihood` objects (PDF)

Here we create the NumCosmo objects necessary for mass fitting using the full photo-z pdf. These are then serialized into experiment files which can either be loaded by and used by custom python scripts or by the numcosmo CLI app.

In [ ]:
for cluster in tqdm(cluster_catalog):
    # Define Cosmology using Planck 2018 parameters (TT, TE, EE+lowE)
    cosmo = nc.HICosmoDEXcdm()

    cosmo.params_set_default_ftype()
    cosmo.omega_x2omega_k()
    cosmo["H0"] = 67.27
    cosmo["Omegab"] = 0.0494
    cosmo["Omegac"] = 0.3166 - 0.0494
    cosmo["w"] = -1.0
    cosmo["Omegak"] = 0.00

    prim = nc.HIPrimPowerLaw.new()
    prim["n_SA"] = 0.9649

    reion = nc.HIReionCamb.new()

    cosmo.param_set_desc("H0", {"fit": False})
    cosmo.param_set_desc("Omegac", {"fit": False})
    cosmo.param_set_desc("Omegab", {"fit": False})
    cosmo.param_set_desc("w", {"fit": False})
    cosmo.param_set_desc("Omegak", {"fit": False})
    prim.param_set_desc("ln10e10ASA", {"fit": False})
    prim.param_set_desc("n_SA", {"fit": False})
    reion.param_set_desc("z_re", {"fit": False})

    cosmo.add_submodel(prim)
    cosmo.add_submodel(reion)

    # Define Halo and WL Models
    dist = nc.Distance.new(6.0)
    halo_mass_summary = nc.HaloCMParam.new(nc.HaloMassSummaryMassDef.CRITICAL, 200.0)
    density_profile = nc.HaloDensityProfileNFW.new(halo_mass_summary)
    surface_mass_density = nc.WLSurfaceMassDensity.new(dist)
    halo_position = nc.HaloPosition.new(dist)

    surface_mass_density.prepare(cosmo)
    halo_position.prepare(cosmo)

    # Set log10MDelta, cDelta, ra, dec as free parameters
    halo_mass_summary.param_set_desc("log10MDelta", {"fit": True})
    halo_mass_summary.param_set_desc("cDelta", {"fit": True})
    halo_position.param_set_desc("ra", {"fit": True})
    halo_position.param_set_desc("dec", {"fit": True})

    ra = cluster["wl_ra"]
    dec = cluster["wl_dec"]
    z = cluster["z"]
    log10mass = cluster["log10M"]
    c = cluster["c_2020"]

    halo_position["ra"] = ra
    halo_position["dec"] = dec
    halo_position["z"] = z

    if not isinstance(c, np.float64):
        c = 4.0

    halo_mass_summary["log10MDelta"] = log10mass
    halo_mass_summary["cDelta"] = c

    # Use minimum and maximum radius from Hamana et al. 2020
    min_radius = 0.3 / cosmo.h()
    max_radius = 3.0 / cosmo.h()
    # Set boundaries for cluster ra and dec
    max_cl_radius = min_radius

    # Delta RA necesary to reach max_radius
    half_box_side = abs(
        fsolve(
            lambda sep: halo_position.projected_radius_from_ra_dec(cosmo, ra + sep, dec)
            - max_radius,
            0.6,
        )[0]
    )

    # Delta RA necesary to reach max_cl_radius
    half_box_side_cl = abs(
        fsolve(
            lambda sep: halo_position.projected_radius_from_ra_dec(cosmo, ra + sep, dec)
            - max_cl_radius,
            0.6,
        )[0]
    )

    halo_position.param_set_desc(
        "ra",
        {
            "lower-bound": float(ra - half_box_side_cl),
            "upper-bound": float(ra + half_box_side_cl),
        },
    )
    # Set dec boundaries taking into account cos(dec) factor
    halo_position.param_set_desc(
        "dec",
        {
            "lower-bound": float(dec - half_box_side_cl * np.cos(np.radians(dec))),
            "upper-bound": float(dec + half_box_side_cl * np.cos(np.radians(dec))),
        },
    )

    # Set log10MDelta boundaries
    halo_mass_summary.param_set_desc(
        "log10MDelta",
        {
            "lower-bound": 10.0,
            "upper-bound": 17.0,
        },
    )

    # Set cDelta boundaries
    halo_mass_summary.param_set_desc(
        "cDelta",
        {
            "lower-bound": 0.01,
            "upper-bound": 30.0,
        },
    )

    ra_min = ra - half_box_side
    ra_max = ra + half_box_side
    dec_min = dec - half_box_side * np.cos(np.radians(dec))
    dec_max = dec + half_box_side * np.cos(np.radians(dec))

    # Define galaxy distribution models: flat position, p(z) obs, Gaussian shape with
    # HSC like bias parametrization and trace ellipticity conventio
    galaxy_position = nc.GalaxySDPositionFlat.new(ra_min, ra_max, dec_min, dec_max)
    galaxy_redshift_obs = nc.GalaxySDObsRedshiftPz.new()
    galaxy_shape = nc.GalaxySDShapeGaussHSC.new(nc.GalaxyWLObsEllipConv.TRACE)

    z_data = nc.GalaxySDObsRedshiftData.new(galaxy_redshift_obs)
    p_data = nc.GalaxySDPositionData.new(galaxy_position, z_data)
    s_data = nc.GalaxySDShapeData.new(galaxy_shape, p_data)

    shear_catalog = Table.read(
        f"clusters/{cluster['wl_name']}/{cluster['wl_name']}_shear_catalog.fits"
    )

    cut_shear_catalog_dict = {
        "ra": [],
        "dec": [],
        "epsilon_obs_1": [],
        "epsilon_obs_2": [],
        "std_shape": [],
        "std_noise": [],
        "m": [],
        "c1": [],
        "c2": [],
    }
    cut_shear_catalog_dict["pz"] = []

    for i in range(len(shear_catalog)):
        radius = halo_position.projected_radius_from_ra_dec(
            cosmo, shear_catalog["ira"][i], shear_catalog["idec"][i]
        )

        # Apply radial cuts from Hamana et al. 2020
        if radius > max_radius or radius < min_radius:
            continue

        # Minimize size of P(z) spline by removing leading and trailing zeros
        lower_index = 0
        upper_index = len(shear_catalog["P(z)"][i]) - 1

        for j in range(len(shear_catalog["P(z)"][i])):
            if shear_catalog["P(z)"][i][j] > 0.0:
                lower_index = j
                break

        for j in range(len(shear_catalog["P(z)"][i]) - 1, -1, -1):
            if shear_catalog["P(z)"][i][j] > 0.0:
                upper_index = j
                break

        if upper_index - lower_index + 1 < 6:
            continue

        xv = ncm.Vector.new(upper_index - lower_index + 1)
        yv = ncm.Vector.new(upper_index - lower_index + 1)

        for j in range(0, upper_index - lower_index + 1):
            xv.set(j, np.array(pz_bins["BINS"])[j + lower_index])
            yv.set(j, shear_catalog["P(z)"][i][j + lower_index])

        pz_spline = ncm.SplineCubicNotaknot.new_full(xv, yv, True)

        pz_spline.prepare()

        # Normalize P(z)
        norm = pz_spline.eval_integ(xv.get(0), xv.get(xv.len() - 1))

        for j in range(0, yv.len()):
            yv.set(j, yv.get(j) / norm)

        pz_spline.prepare()

        # P-CUT FROM UMETSU 2020 AND MEDEZINSKI 2018B
        p_cut = 0.98
        delta_z = 0.2
        z_min = halo_position["z"] + delta_z
        z_max = 2.5

        if z_min >= xv.get(xv.len() - 1):
            continue

        # Use photo-z MC estimate to apply upper redshift cut
        if shear_catalog["photoz_mc"][i] > z_max:
            continue

        if pz_spline.eval_integ(max(xv.get(0), z_min), xv.get(xv.len() - 1)) < p_cut:
            continue

        cut_shear_catalog_dict["pz"].append(pz_spline)
        cut_shear_catalog_dict["ra"].append(shear_catalog["ira"][i])
        cut_shear_catalog_dict["dec"].append(shear_catalog["idec"][i])
        cut_shear_catalog_dict["epsilon_obs_1"].append(
            shear_catalog["ishape_hsm_regauss_e1"][i]
        )
        cut_shear_catalog_dict["epsilon_obs_2"].append(
            shear_catalog["ishape_hsm_regauss_e2"][i]
        )
        cut_shear_catalog_dict["std_shape"].append(
            shear_catalog["ishape_hsm_regauss_derived_rms_e"][i]
        )
        cut_shear_catalog_dict["std_noise"].append(
            shear_catalog["ishape_hsm_regauss_derived_sigma_e"][i]
        )
        cut_shear_catalog_dict["m"].append(
            shear_catalog["ishape_hsm_regauss_derived_shear_bias_m"][i]
        )
        cut_shear_catalog_dict["c1"].append(
            shear_catalog["ishape_hsm_regauss_derived_shear_bias_c1"][i]
        )
        cut_shear_catalog_dict["c2"].append(
            shear_catalog["ishape_hsm_regauss_derived_shear_bias_c2"][i]
        )
        cut_shear_catalog_dict["z"].append(shear_catalog["photoz_mc"][i])

    print(
        f"Cluster: {cluster['wl_name']}, Number of galaxies: {len(cut_shear_catalog_dict['ra'])}"
    )

    # Create observables object
    # Note that we are using TRACE ellipticity convention and
    # celestial coordinates
    wl_obs = nc.GalaxyWLObs.new(
        nc.GalaxyWLObsEllipConv.TRACE,
        nc.GalaxyWLObsCoord.CELESTIAL,
        len(cut_shear_catalog_dict["ra"]),
        s_data.required_columns(),
    )

    for i in range(len(cut_shear_catalog_dict["ra"])):
        for key, value in cut_shear_catalog_dict.items():
            if key == "pz":
                wl_obs.set_pz(i, cut_shear_catalog_dict["pz"][i])
            else:
                wl_obs.set(key, i, value[i])

    # Create DataClusterWL object (we do not set radial cuts on the likelihood)
    data_cluster = nc.DataClusterWL.new()
    data_cluster.set_obs(wl_obs)
    data_cluster.set_init(True)

    mset = ncm.MSet.new_array(
        [
            cosmo,
            density_profile,
            surface_mass_density,
            halo_position,
            galaxy_position,
            galaxy_redshift_obs,
            galaxy_shape,
        ]
    )
    dataset = ncm.Dataset.new_array([data_cluster])
    likelihood = ncm.Likelihood.new(dataset)

    mset.prepare_fparam_map()

    experiment = ncm.ObjDictStr()

    experiment.set("likelihood", likelihood)
    experiment.set("model-set", mset)

    ser = ncm.Serialize.new(ncm.SerializeOpt.CLEAN_DUP)

    ser.to_binfile(
        dataset,
        f"clusters/{cluster['wl_name']}/{cluster['wl_name']}_experiment_pdf.dataset.gvar",
    )
    ser.dict_str_to_yaml_file(
        experiment,
        f"clusters/{cluster['wl_name']}/{cluster['wl_name']}_experiment_pdf.yaml",
    )